In [ ]:
#naive
import numpy as nm 
import matplotlib.pyplot as mtp 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 

In [ ]:
dataset = pd.read_csv('naive.csv') 
print("Sample Dataset - \n",dataset,"\n")

Sample Dataset - 
     Day   Outlook Temperature Humidity    Wind PlayTennis
0    D1     Sunny         Hot     High    Weak         No
1    D2     Sunny         Hot     High  Strong         No
2    D3  Overcast         Hot     High    Weak        Yes
3    D4      Rain        Mild     High    Weak        Yes
4    D5      Rain        Cool   Normal    Weak        Yes
5    D6      Rain        Cool   Normal  Strong         No
6    D7  Overcast        Cool   Normal  Strong        Yes
7    D8     Sunny        Mild     High    Weak         No
8    D9     Sunny        Cool   Normal    Weak        Yes
9   D10      Rain        Mild   Normal    Weak        Yes
10  D11     Sunny        Mild   Normal  Strong        Yes
11  D12  Overcast        Mild     High  Strong        Yes
12  D13  Overcast         Hot   Normal    Weak        Yes
13  D14      Rain        Mild     High  Strong         No 



In [ ]:

le_outlook = LabelEncoder() 
dataset['outlook_n'] = le_outlook.fit_transform(dataset['Outlook']) 
le_temperature = LabelEncoder() 
dataset['temperature_n'] = le_temperature.fit_transform(dataset['Temperature']) 
le_humidity = LabelEncoder() 
dataset['humidity_n'] = le_humidity.fit_transform(dataset['Humidity']) 
le_wind = LabelEncoder() 
dataset['wind_n'] = le_wind.fit_transform(dataset['Wind'])
print("Given Data after Encoding - \n",dataset,"\n") 

Given Data after Encoding - 
     Day   Outlook Temperature Humidity    Wind PlayTennis  outlook_n  \
0    D1     Sunny         Hot     High    Weak         No          2   
1    D2     Sunny         Hot     High  Strong         No          2   
2    D3  Overcast         Hot     High    Weak        Yes          0   
3    D4      Rain        Mild     High    Weak        Yes          1   
4    D5      Rain        Cool   Normal    Weak        Yes          1   
5    D6      Rain        Cool   Normal  Strong         No          1   
6    D7  Overcast        Cool   Normal  Strong        Yes          0   
7    D8     Sunny        Mild     High    Weak         No          2   
8    D9     Sunny        Cool   Normal    Weak        Yes          2   
9   D10      Rain        Mild   Normal    Weak        Yes          1   
10  D11     Sunny        Mild   Normal  Strong        Yes          2   
11  D12  Overcast        Mild     High  Strong        Yes          0   
12  D13  Overcast         Hot   No

In [ ]:
x = dataset[['outlook_n','temperature_n','humidity_n','wind_n']] 
print("X - Values\n",x,"\n") 
y = dataset['PlayTennis'] 
print("Y - Values\n",y,"\n")


X - Values
     outlook_n  temperature_n  humidity_n  wind_n
0           2              1           0       1
1           2              1           0       0
2           0              1           0       1
3           1              2           0       1
4           1              0           1       1
5           1              0           1       0
6           0              0           1       0
7           2              2           0       1
8           2              0           1       1
9           1              2           1       1
10          2              2           1       0
11          0              2           0       0
12          0              1           1       1
13          1              2           0       0 

Y - Values
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object 



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 0) 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(x, y) 
y_pred = gnb.predict(x_test) 
print("Testing values for play tennis\n",y_test) 
print("Predicted values for play tennis",y_pred) 

Testing values for play tennis
 8    Yes
6    Yes
4    Yes
Name: PlayTennis, dtype: object
Predicted values for play tennis ['Yes' 'Yes' 'Yes']


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

1.0

In [ ]:
import pandas as pd
import numpy as np
import math
import random
import warnings
warnings.filterwarnings("ignore")
def load_csv(filename):
    return pd.read_csv(filename)
 
def str_column_to_int(dataset):
    for column in dataset.columns:
        if dataset[column].dtype == np.object:
            dataset[column] = dataset[column].astype('category').cat.codes
    return dataset
 
def split_dataset(dataset, split_ratio):
    train_size = int(len(dataset) * split_ratio)
    train_set = dataset.sample(n=train_size)
    test_set = dataset.drop(train_set.index)
    return [train_set, test_set]
 
def separate_by_class(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset.iloc[i]
        if (vector.iloc[-1] not in separated):
            separated[vector.iloc[-1]] = []
        separated[vector.iloc[-1]].append(vector)
    return separated
 
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)
 
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries
 
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    for class_value, instances in separated.items():
        summaries[class_value] = summarize(instances)
    return summaries
 
def calculate_probability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
 
def calculate_class_probabilities(summaries, input_vector):
    probabilities = {}
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = 1
        for i in range(len(class_summaries)):
            mean, stdev = class_summaries[i]
            x = input_vector[i]
            probabilities[class_value] *= calculate_probability(x, mean, stdev)
    return probabilities
 
def predict(summaries, input_vector):
    probabilities = calculate_class_probabilities(summaries, input_vector)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label
 
def get_predictions(summaries, test_set):
    predictions = []
    for i in range(len(test_set)):
        result = predict(summaries, test_set.iloc[i])
        predictions.append(result)
    return predictions
 
def get_accuracy(test_set, predictions):
    correct = 0
    for i in range(len(test_set)):
        if test_set.iloc[i,-1] == predictions[i]:
            correct += 1
    return (correct / float(len(test_set))) * 100.0
 
def main():
    filename = 'naive.csv'
    split_ratio = 0.8
    dataset = load_csv(filename)
    dataset = str_column_to_int(dataset)
    training_set, test_set = split_dataset(dataset, split_ratio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(training_set), len(test_set)))
    # prepare model
    summaries = summarize_by_class(training_set)
    # test model
    predictions = get_predictions(summaries, test_set)
    accuracy = get_accuracy(test_set, predictions)
    print('Accuracy: {0}%\n\n\n\n'.format(accuracy))

main()


Split 14 rows into train=11 and test=3 rows
Accuracy: 100.0%




